In [1]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
from preprocess_days_stats import preprocess_cumulative_stats, preprocess_match_days
import warnings
# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")

giorni_cumulativi=1
numero_colonne='all'

df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A\csv_serie_a")
df_Serie_A_5days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)


Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (18).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!

le squadre sono uguali:
 [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]
preprocess finished


In [2]:
pd.set_option('display.max_columns', None)
df_Serie_A_5days.head()

,div,giornata,stagione,date,hometeam,awayteam,home_total_points,home_result,home_result_1,away_total_points,away_result,away_result_1,ft_home_goals,home_last_1_days_ft_goals,ft_away_goals,away_last_1_days_ft_goals,ft_result,ht_home_goals,home_last_1_days_ht_goals,ht_away_goals,away_last_1_days_ht_goals,home_last_1_days_ft_goals_conceded,away_last_1_days_ft_goals_conceded,ht_results,home_shots,home_last_1_days_shots,away_shots,away_last_1_days_shots,home_shots_targ,home_last_1_days_shots_target,away_shots_targ,away_last_1_days_shots_target,home_fouls,home_last_1_days_fouls_done,away_fouls,away_last_1_days_fouls_done,home_corners,home_last_1_days_corners_obtained,away_corners,away_las_t1_days_corners_obtained,home_yellow,home_last_1_days_yellows,away_yellow,away_last_1_days_yellows,home_red,home_last_1_days_reds,away_red,away_last_1_days_reds,home_win_odds,draw_odds,away_win_odds
0,I1,1,2005/2006,2005-08-27,Fiorentina,Sampdoria,0.0,W,###,0.0,L,###,2.0,0.0,1.0,0.0,H,2.0,0.0,0.0,0.0,0.0,0.0,H,15.0,0.0,9.0,0.0,9.0,0.0,4.0,0.0,18.0,0.0,23.0,0.0,9.0,0.0,3.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.20,2.87,3.25
1,I1,1,2005/2006,2005-08-27,Livorno,Lecce,0.0,W,###,0.0,L,###,2.0,0.0,1.0,0.0,H,1.0,0.0,1.0,0.0,0.0,0.0,D,17.0,0.0,6.0,0.0,7.0,0.0,5.0,0.0,27.0,0.0,21.0,0.0,9.0,0.0,2.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.90,2.90,4.00
2,I1,1,2005/2006,2005-08-28,Ascoli,Milan,0.0,D,###,0.0,D,###,1.0,0.0,1.0,0.0,D,0.0,0.0,0.0,0.0,0.0,0.0,D,8.0,0.0,16.0,0.0,3.0,0.0,9.0,0.0,22.0,0.0,16.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.50,3.75,1.40
3,I1,1,2005/2006,2005-08-28,Inter,Treviso,0.0,W,###,0.0,L,###,3.0,0.0,0.0,0.0,H,1.0,0.0,0.0,0.0,0.0,0.0,H,16.0,0.0,7.0,0.0,9.0,0.0,3.0,0.0,13.0,0.0,20.0,0.0,6.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.16,5.50,13.00
4,I1,1,2005/2006,2005-08-28,Juventus,Chievo,0.0,W,###,0.0,L,###,1.0,0.0,0.0,0.0,H,1.0,0.0,0.0,0.0,0.0,0.0,H,16.0,0.0,2.0,0.0,7.0,0.0,0.0,0.0,16.0,0.0,12.0,0.0,6.0,0.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.25,5.00,13.00


# Capiamo la precisione da battere

In [3]:
conditions_1 = [
      ( (df_Serie_A_5days[f'home_win_odds'] <= df_Serie_A_5days['draw_odds']) & (df_Serie_A_5days[f'home_win_odds'] <= df_Serie_A_5days[f'away_win_odds'] )), 
      ( (df_Serie_A_5days['draw_odds'] <= df_Serie_A_5days[f'away_win_odds']) & (df_Serie_A_5days['draw_odds'] <= df_Serie_A_5days[f'home_win_odds'] )), 
      ( (df_Serie_A_5days[f'away_win_odds'] <= df_Serie_A_5days['draw_odds']) & (df_Serie_A_5days[f'away_win_odds'] <= df_Serie_A_5days[f'home_win_odds'] ))
      ]
values=['W','D','L']

precision = len(np.select(conditions_1, values)[df_Serie_A_5days[f'home_result']==np.select(conditions_1, values)])/len(df_Serie_A_5days) 
precision

0.5459770114942529

la precision da battere è 0.55

# Estraggo le colonne utili al mio modello

In [4]:
all_columns = ['giornata', 'stagione','hometeam', 'awayteam','home_total_points','home_result', 'away_total_points', f'home_last_{giorni_cumulativi}_days_ft_goals',
               f'away_last_{giorni_cumulativi}_days_ft_goals',f'home_last_{giorni_cumulativi}_days_ht_goals',f'away_last_{giorni_cumulativi}_days_ht_goals', f'away_last_{giorni_cumulativi}_days_shots',
       f'home_last_{giorni_cumulativi}_days_ft_goals_conceded',f'away_last_{giorni_cumulativi}_days_ft_goals_conceded',
       f'home_last_{giorni_cumulativi}_days_shots',f'home_last_{giorni_cumulativi}_days_shots_target',
       f'away_last_{giorni_cumulativi}_days_shots_target',f'home_last_{giorni_cumulativi}_days_fouls_done',f'away_last_{giorni_cumulativi}_days_fouls_done',f'home_last_{giorni_cumulativi}_days_corners_obtained',
       f'away_last_{giorni_cumulativi}_days_corners_obtained',f'home_last_{giorni_cumulativi}_days_yellows',f'away_last_{giorni_cumulativi}_days_yellows',
       f'home_last_{giorni_cumulativi}_days_reds', f'away_last_{giorni_cumulativi}_days_reds']

less_columns = ['stagione','hometeam', 'awayteam','home_total_points','home_result', 'away_total_points',f'home_last_{giorni_cumulativi}_days_ft_goals',
       f'away_last_{giorni_cumulativi}_days_ft_goals',f'home_last_{giorni_cumulativi}_days_ft_goals_conceded',f'away_last_{giorni_cumulativi}_days_ft_goals_conceded',f'home_last_{giorni_cumulativi}_days_shots',
       f'away_last_{giorni_cumulativi}_days_shots',f'home_last_{giorni_cumulativi}_days_yellows',f'away_last_{giorni_cumulativi}_days_yellows']

few_columns = ['hometeam', 'awayteam','home_total_points','home_result', 'away_total_points',f'home_last_{giorni_cumulativi}_days_ft_goals',
               f'away_last_{giorni_cumulativi}_days_ft_goals',f'home_last_{giorni_cumulativi}_days_ft_goals_conceded',
               f'away_last_{giorni_cumulativi}_days_ft_goals_conceded',f'home_last_{giorni_cumulativi}_days_shots',
               f'away_last_{giorni_cumulativi}_days_shots']


if numero_colonne == 'all':
       colonne = all_columns
elif numero_colonne == 'less':
       colonne = less_columns
else:
       colonne=few_columns


for day in range(giorni_cumulativi):
       colonne.append(f'home_result_{day+1}')
       colonne.append(f'away_result_{day+1}')

In [5]:
len(all_columns)

27

In [6]:
df_stats_serie_A = df_Serie_A_5days[colonne]

pd.set_option('display.max_columns', None)
len(df_stats_serie_A), display(df_stats_serie_A.head())

KeyError: "['away_last_1_days_corners_obtained'] not in index"

# Test valid train split

Il test train split verrà fatto in maniera casuale, mettendo nel validation dataset il 0.5% del dataframe e nel test l'1% del dataframe.
Non consideremo le partite come time series. Questo verrà fatto più avanti.

In [ ]:
from sklearn.model_selection import train_test_split

#creo il train valid test
X_train_df, X_test_df, Train_labels, Test_labels = train_test_split(df_stats_serie_A.drop(f'home_result', axis=1), df_stats_serie_A[f'home_result'],
                                                                     test_size=0.1, random_state=42)

X_train_df, X_valid_df, Train_labels, Valid_labels = train_test_split(X_train_df, Train_labels, test_size=0.055, random_state=42)


len(X_train_df), len(X_valid_df), len(X_test_df)

(5919, 345, 696)

In [ ]:
all_numerical_columns= [f'home_total_points',f'away_total_points',f'home_last_{giorni_cumulativi}_days_ft_goals',f'away_last_{giorni_cumulativi}_days_ft_goals',f'home_last_{giorni_cumulativi}_days_ht_goals',
       f'away_last_{giorni_cumulativi}_days_ht_goals', f'home_last_{giorni_cumulativi}_days_ft_goals_conceded',f'away_last_{giorni_cumulativi}_days_ft_goals_conceded',
       f'home_last_{giorni_cumulativi}_days_shots',f'home_last_{giorni_cumulativi}_days_shots_target',f'away_last_{giorni_cumulativi}_days_shots',f'away_last_{giorni_cumulativi}_days_shots_target',
       f'home_last_{giorni_cumulativi}_days_fouls_done',f'away_last_{giorni_cumulativi}_days_fouls_done',f'home_last_{giorni_cumulativi}_days_corners_obtained',
       f'away_last_{giorni_cumulativi}_days_corners_obtained',f'home_last_{giorni_cumulativi}_days_yellows',f'away_last_{giorni_cumulativi}_days_yellows',f'home_last_{giorni_cumulativi}_days_reds', f'away_last5_days_reds']

less_numerical_columns = [f'home_total_points',f'away_total_points',f'home_last_{giorni_cumulativi}_days_ft_goals',f'away_last_{giorni_cumulativi}_days_ft_goals',f'home_last_{giorni_cumulativi}_days_ft_goals_conceded',
                          f'away_last_{giorni_cumulativi}_days_ft_goals_conceded',f'away_last_{giorni_cumulativi}_days_shots',f'home_last_{giorni_cumulativi}_days_shots',
                          f'home_last_{giorni_cumulativi}_days_yellows',f'away_last_{giorni_cumulativi}_days_yellows']

few_numerical_columns = [f'home_total_points',f'away_total_points',f'home_last_{giorni_cumulativi}_days_ft_goals',f'away_last_{giorni_cumulativi}_days_ft_goals',f'home_last_{giorni_cumulativi}_days_ft_goals_conceded',
                          f'away_last_{giorni_cumulativi}_days_ft_goals_conceded',f'away_last_{giorni_cumulativi}_days_shots',f'home_last_{giorni_cumulativi}_days_shots']


all_categorical_columns = ['giornata', 'stagione',f'hometeam', f'awayteam']
less_categorical_columns = ['stagione',f'hometeam', f'awayteam']
few_categorical_columns = [f'hometeam', f'awayteam']

if numero_colonne == 'all':
       categorical_columns = all_categorical_columns
       numerical_columns = all_numerical_columns
elif numero_colonne == 'less':
       categorical_columns = less_categorical_columns
       numerical_columns = less_numerical_columns
else:
       categorical_columns=few_categorical_columns
       numerical_columns = few_numerical_columns


for day in range(giorni_cumulativi):
       categorical_columns.append(f'home_result_{day+1}')
       categorical_columns.append(f'away_result_{day+1}')

In [ ]:
# generalmente le reti neurali preferiscono la normalizzazione
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

#Creo un column transformer, rende più facile normalizzare il dataframe
ct = make_column_transformer(
(MinMaxScaler(), numerical_columns), #normalizza queste colonne con minmax
(OneHotEncoder(handle_unknown='ignore'), categorical_columns) ,# le colonne da one hot
    #encodare, tutte le altre le ignora grazie al comando handle_unknown
sparse_threshold=0  
)

ct.fit(X_train_df)

# trasformo train e test
X_train_norm = ct.transform(X_train_df)
X_valid_norm  = ct.transform(X_valid_df)
X_test_norm = ct.transform(X_test_df)

# Crea un OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
# Addestra l'OrdinalEncoder su Train_labels e applica la trasformazione
Train_labels_encoded = ordinal_encoder.fit_transform(np.array(Train_labels).reshape(-1, 1))

Train_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Train_labels).reshape(-1, 1)))
Valid_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Valid_labels).reshape(-1, 1)))
Test_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Test_labels).reshape(-1, 1)))

#visualizziamo
Train_labels_encoded, np.array(Train_labels)

(array([2., 2., 0., ..., 2., 1., 2.]),
 array(['W', 'W', 'D', ..., 'W', 'L', 'W'], dtype=object))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score

#definisco il mio modello di training con k=5
knn_model = KNeighborsClassifier(n_neighbors=35)
knn_model.fit(X_train_norm, Train_labels_encoded) #alleno il mio modello sul mio train dataset

y_pred = knn_model.predict(X_train_norm)
print(round(accuracy_score(Train_labels_encoded, y_pred),2)) #stampa solo l'accuracy
classification_report(Train_labels_encoded, y_pred)

0.52


'              precision    recall  f1-score   support\n\n         0.0       0.43      0.15      0.22      1563\n         1.0       0.52      0.41      0.46      1705\n         2.0       0.54      0.82      0.65      2651\n\n    accuracy                           0.52      5919\n   macro avg       0.50      0.46      0.44      5919\nweighted avg       0.50      0.52      0.48      5919\n'

In [ ]:
best_accuracy=0
for vicini in range(3,40): #prendiamo diversi numeri di vicini possibili
    knn_model = KNeighborsClassifier(n_neighbors=3*vicini) #addestriamo il modello con questo numero di vicini 
    knn_model.fit(X_train_norm, Train_labels_encoded) 
    y_valid_test=knn_model.predict(X_valid_norm)
    accuracy=round(accuracy_score(Valid_labels_encoded, y_valid_test),2)

    if accuracy > best_accuracy: #cerco di capire quale numero di vicini restituisce il risultato migliore
        best_vicini = 5*vicini
        best_accuracy = accuracy

print(f"il modello migliore si ha con {best_vicini} vicini, con un'accuracy di {best_accuracy}")

il modello migliore si ha con 120 vicini, con un'accuracy di 0.51
